In [1]:
! pip install nltk


In [2]:
%set_env JAVAHOME=/home/admin/java/jdk1.8.0_251
from nltk.tag import StanfordNERTagger
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

import pandas as pd
import timeit
import re

from ner_tool import NerTool

env: JAVAHOME=/home/admin/java/jdk1.8.0_251


In [3]:
# Load NER
ner_model = '/home/admin/text_processing/ner/model/idner-model-20k-mdee.ser.gz'
ner_app = '/home/admin/stanford-ner/stanford-ner.jar'
ner_tool = NerTool(ner_model, ner_app)
ner_tagger = StanfordNERTagger(ner_model, ner_app, encoding='utf-8')

In [4]:
# Retrieve gold standard dataset
f_gold = '/home/admin/text_processing/anotated_data/gold_standard/anotated/sample_banjir_gold_dataset.xlsx'
df_gold = pd.read_excel(f_gold)

In [5]:
# re keywords based on file name
re_keywords = ''
if 'angin_topan' in f_gold:
    re_keywords = r'badai|puting beliung|angin topan|tornado|angin kencang'
elif 'banjir' in f_gold:
    re_keywords = r'banjir'
elif 'erupsi' in f_gold:
    re_keywords = r'vulkanik|erupsi|letusan|awan panas|lava'
elif 'gempa' in f_gold:
    re_keywords = r'gempa|tektonik'
elif 'karhutla'in f_gold:
    re_keywords = r'kebakaran hutan|kebakaran lahan|titik panas'
elif 'kekeringan'in f_gold:
    re_keywords = r'kekeringan'
elif 'longsor'in f_gold:
    re_keywords = r'longsor'
elif 'tsunami'in f_gold:
    re_keywords = r'tsunami'
else:
    re_keyowrds = 'XXXXXXXXXXXXXXXX'

In [6]:
df_out = df_gold[[ 'id', ]].copy()
df_out['raw_ner'] = ''
df_out['exec_time'] = ''

In [10]:
df_data = df_gold[df_gold.label == 'Y'][['id', 'content']]

for index, row in df_data[:3].iterrows():
    row_id = row['id']
    row_text = row['content']
    
    start_time = timeit.default_timer()
    sent_tokens = sent_tokenize(row_text)
    sent_valid = [ l for l in sent_tokens if None != re.search(re_keywords, l, flags=re.IGNORECASE) ]
    w_tokens = word_tokenize(' '.join(sent_valid))

    row_loc = ner_tool.parse_text(w_tokens)
    row_loc = ", ".join(row_loc)
    exec_time = timeit.default_timer() - start_time
    
    df_out.loc[df_out['id'] == row_id, 'raw_ner'] = row_loc
    df_out.loc[df_out['id'] == row_id, 'exec_time'] = exec_time

In [11]:
df_out

,id,raw_ner,exec_time
0,banjir_0001,,
1,banjir_0002,"Kecamatan, Aceh, Leuser",2.20819
2,banjir_0003,,
3,banjir_0004,,
4,banjir_0005,,
...,...,...,...
1239,banjir_1240,,
1240,banjir_1241,,
1241,banjir_1242,,
1242,banjir_1243,,


In [8]:
#loc_tag = ner_tagger.parse_text(text)

In [9]:
#loc_tag